In [1]:
import tensorflow as tf

print(tf.__version__)

2.1.0


In [5]:
import numpy as np
import tensorflow as tf

# 랜덤값을 하나 얻어온다
random1 = np.random.rand(2,2)
print(random1)

# Tensorflow
random2 = tf.random.normal([1], dtype=tf.float32)
print(random2)  # Tensor가 출력
# TF 1.x버전에서는 node를 실행시키려면 session이 있어야했다.
# 하지만 2.x버전부터는 session 없이 즉시 실행 가능하다. (Eager Execution)

print(random2.numpy())

[[0.55630135 0.18739828]
 [0.52411839 0.80281788]]
tf.Tensor([1.9922442], shape=(1,), dtype=float32)
[1.9922442]


In [6]:
import tensorflow as tf

a = tf.constant(10, dtype=tf.float32)
b = tf.constant(20, dtype=tf.float32)

c = a + b

print('c의 값은 : {}'.format(c.numpy()))

d= 30.0

tensor_d = tf.convert_to_tensor(d)

print((c + tensor_d).numpy())

c의 값은 : 30.0
60.0


In [ ]:
import tensorflow as tf

W = tf.Variable(tf.random.normal([1]), name='weight')

# 기존에는 tf.variable()을 이용해서 변수를 만들면 사용하기 전에 반드시 초기화를 진행해야 했다.
# TF 2.0부터는 초기화를 안해도 된다.
print(W.numpy())

In [ ]:
# tensorflow graph에 입력을 주는 부분이 없어졌다.
# 기존에는 graph에 데이터를 밀어넣기 위해 placeholder를 이용했다.
# Eager Execution에 의해서 이제는 placeholder가 필요 없게 됐다.
# placeholder는 삭제되었다.

# Lazy execution은 더이상 x eager execution

In [8]:
# tensorflow의 keras를 이용하여 model을 생성해 보자

import tensorflow as tf
# from tensorflow.keras.models import Sequential

# model = Sequential()  # keras model 생성
model = tf.keras.models.Sequential()

# model을 만들었으니 그 다음에는 layer를 만들어야 한다.
model.add(tf.keras.layers.Flatten(input_shape=(2,)))  # layer를 추가
model.add(tf.keras.layers.Dense(3, activation='softmax'))

# model compile 과정
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=['accuracy'])

def my_loss:
    pass

# model 학습
model.fit(x_data_train, t_data_train, epochs=100, batch_size=100, validation_split=0.3)



In [11]:
## tensorflow 2.1을 이용하여 Ozone 예제를 다시 구현해보자
## Multiple Linear Regression

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential  # Model
from tensorflow.keras.layers import Flatten, Dense  # Layer
from tensorflow.keras.optimizers import SGD  # Optimizer
from sklearn.preprocessing import MinMaxScaler  # Normmalization
from scipy import stats   # 이상치 처리

# Raw Data Lodaing
df = pd.read_csv('./data/ozone.csv')

# 결측치 확인 및 처리
# 데이터가 충분히 많고 결측치가 적으면 삭제가 답이다. 하지만 일반적으로 결측치를 삭제하면 데이터가 유실
print(df.shape)
print(df.isnull().sum())

(153, 6)
Ozone      37
Solar.R     7
Wind        0
Temp        0
Month       0
Day         0
dtype: int64


In [13]:
# 우리 KNN의 사용법에 대해서 알아보자
# sklearn을 이용해서 알아보자

# BMI 예제를 이용해서 학습한 후 정확도를 측정해보자

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Raw Data Loading
df = pd.read_csv('./data/bmi/bmi.csv', skiprows=3)

# data split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df[['height','weight']], df['label'], test_size=0.3, random_state=0)

# Normalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

# Logistic Regression
model = LogisticRegression()
model.fit(x_data_train_norm, t_data_train)
print(model.score(x_data_test_norm, t_data_test))


# KNN 을 이용한 분류
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(x_data_train_norm, t_data_train)
print(knn_model.score(x_data_test_norm, t_data_test))

0.9845
0.998


In [22]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
import warnings

warnings.filterwarnings(action='ignore')   # warning 출력을 하지 않음

# Raw Data Loading
df = pd.read_csv('./data/ozone.csv')

training_data = df

x_data = training_data[['Solar.R', 'Wind', 'Temp']]
t_data = training_data['Ozone']

# 결측치 확인
# 1. 독립변수에 대한 결측치 처리부터 하고 가자
# Solar.R의 7개의 결측치를 median으로 처리할 거다
for col in x_data.columns:
    col_median = np.nanmedian(x_data[col])
    x_data[col].loc[x_data[col].isnull()] = col_median

# 2. 독립변수에 대한 이상치를 검출 한 후 이상치는 mean처리할 거다
zscore = 1.8

for col in x_data.columns:
    outliers = x_data[col][np.abs(stats.zscore(x_data[col])) > zscore ]
#     print(outliers)
    col_mean = np.mean(x_data.loc[~x_data[col].isin(outliers),col])
    x_data.loc[~x_data[col].isin(outliers),col] = col_mean

# 3. 정규화를 진행하자

scaler_x = MinMaxScaler()
scaler_t = MinMaxScaler()

scaler_x.fit(x_data.values)
scaler_t.fit(t_data.values.reshape(-1,1))

x_data_norm = scaler_x.transform(x_data)
t_data_norm = scaler_t.transform(t_data.values.reshape(-1,1)).ravel()

# 4. 종속변수(Ozone)에 대한 결측치는 KNN을 이용해서 예측값으로 imputation할거다
#   학습에 사용될 x_data_train_norm, t_data_train_norm을 구해야한다.
x_data_train_norm = x_data_norm[~np.isnan(t_data_norm)]
t_data_train_norm = t_data_norm[~np.isnan(t_data_norm)]

knn_regressor = KNeighborsRegressor(n_neighbors=2)
knn_regressor.fit(x_data_train_norm, t_data_train_norm)

# knn_predict
knn_predict = knn_regressor.predict(x_data_norm[np.isnan(t_data_norm)])
t_data_norm[np.isnan(t_data_norm)] = knn_predict

# 최종 데이터를 생성했다.

##########################
# 학습을 진행해보자 Linear Regression

from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD      # stokastic Gradient Descent

test_data = [[310, 15, 80]]

# sklearn
sklearn_model = LinearRegression()
sklearn_model.fit(x_data_norm, t_data_norm)
result = sklearn_model.predict(scaler_x.transform(test_data)).reshape(-1,1)

scaled_result = scaler_t.inverse_transform(result)
print('sklearn: {}'.format(scaled_result))

# Tensorflow 2.x
# 모델생성
keras_model = Sequential()
# 레이어 추가
keras_model.add(Flatten(input_shape=(3,)))   # input layer
keras_model.add(Dense(1, activation='linear'))  # output layer

# complie
keras_model.compile(optimizer=SGD(learning_rate=1e-2),
                    loss='mse')

# 학습
keras_model.fit(x_data_norm, t_data_norm, epochs=5000, verbose=0) 
#verbose는 epoch마다 loss찍어줌

# prediction
result = keras_model.predict(scaler_x.transform(test_data)).reshape(-1,1)

scaled_result = scaler_t.inverse_transform(result)
print('tensorflow:{}'.format(scaled_result))

sklearn: [[29.58893827]]
tensorflow:[[29.775517]]


In [ ]:
## LogisticRegression에 대해서 sklearn과 Tensorflow 2.x 구현

## titanic => logistic 문제 (결측치 다수)
## feature engineering

## 데이터를 완전히 준비하고 구현해보자

%reset

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats

# Raw Data Loading
df = pd.read_csv('./data/titanic/train.csv')

df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Fare'], axis=1, inplace=False)

df['Family'] = df['SibSp'] + df['Parch']
df.drop(['SibSp', 'Parch'], axis=1, inplace=True)

sex_dict = {'male': 0 ,'female' : 1}
df['Sex'] = df['Sex'].map(sex_dict)
embarked_dict = {'S' : 0 , 'C' : 1, 'Q' :2}
df['Embarked'] = df['Embarked'].map(embarked_dict)

## 결측치 처리
df.loc[df['Age'].isnull(),'Age'] = np.nanmedian(df['Age'].values)


############

# sklearn으로 모델 구현
sklearn_model = LogisticRegression()
sklearn_model.fit(x_data_train_norm, t_data_train)
sklearn_result = sklearn_model.score(x_data_test_norm, t_data_test)

print('sklearn 정확도 : {}'.format(sklearn_result))

# tensorflow 2.x로 구현
keras_model = Sequential()

keras_model.add(Flatten(input_shape=(x_data_train_norm.shape[1],)))
keras_model.add(Dense(1, activation='sigmoid'))

keras_model.compile(optimizers=SGD(learning_rate=1e-3),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

result= keras_model.fit(x_data_train_nrom,
                        t_data_train,
                        epochs=1000,
                        verbose=1,
                        validation_split=0.3)

keras_result = keras_model.evalutate(x_data_test_norm, t_data_test)
print('TF : {}'.format(keras_result))

In [ ]:
import matplotlib.pyplot as plt

print(result.history.keys())

plt.plot(result.history['accuracy'], color='b')
plt.plot(result.history['val_accuracy'], color='r')

plt.show()

In [24]:
## Multinomial Classification에 대해서 구현해보자
# MNIST 예제를 이용해서 구현해보자

%reset
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD 

# Raw Data Loading
df = pd.read_csv('./data/digit-recognizer/train.csv')

# 결측치 이상치는 없다. Feature Engineering 필요없다.

# 독립변수와 종속변수 분리
x_data = df.drop('label', axis=1, inplace=False)
t_data = df['label']   # one-hot encoding 일단 안함

# Normalization
scaler = MinMaxScaler()
scaler.fit(x_data)
x_data_norm = scaler.transform(x_data)

# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(x_data_norm, t_data, test_size=0.3, random_state=0)

# 데이터 준비가 끝났다. 학습 진행
sklearn_model = LogisticRegression(solver='saga')
# solver라는 개념이 있는데 default로 사용되는건 lbfgs라는 놈
# lbfgs: 작은 데이터셋에 좋다. 데이터량이 많으면 성능이 별로다.
# 데이터량이 많은 경우 sag(Stochastic Average Gradient Descent)를 사용하면 더 좋다
# 일반적으로 또 이걸 개량한 saga를 더 많이 이용한다.

sklearn_model.fit(x_data_train, t_data_train)  # 학습진행
print('sklearn result :')
print(classification_report(t_data_test,
                            sklearn_model.predict(x_data_test)))

# TF 2.0 구현
keras_model = Sequential()
keras_model.add(Flatten(input_shape=(x_data_train.shape[1],)))
keras_model.add(Dense(10, activation='softmax'))

keras_model.compule(optimizer=SGD(learning_rate=1e-2),
                    loss='sparse_categorical_crossentropy',
                    metrics=['sparse_categorical_accuracy'])

history = keras_model.fit(x_data_train,
                          t_data_train,
                          epochs=500,
                          batch_size=100,
                          verbose=0,
                          validation_split=0.3)

print(keras_model.evaluate(x_data_test, t_data_test))
print('tensorflow result :')
print(classification_report(t_data_test,
                            keras_model.predict(x_data_test)))

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
sklearn result :
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1242
           1       0.95      0.97      0.96      1429
           2       0.92      0.90      0.91      1276
           3       0.91      0.90      0.91      1298
           4       0.92      0.92      0.92      1236
           5       0.88      0.88      0.88      1119
           6       0.93      0.96      0.94      1243
           7       0.94      0.93      0.94      1334
           8       0.89      0.88      0.88      1204
           9       0.89      0.89      0.89      1219

    accuracy                           0.92     12600
   macro avg       0.92      0.92      0.92     12600
weighted avg       0.92      0.92      0.92     12600



In [ ]:
# print(classification_report(t_data_test,
#                             (tf.argmax(keras_model.predict(x_data_test), axis=1)).numpy())

# (tf.argmax(keras_model.predict(x_data_test), axis=1)).numpy()

import matplotlib.pyplot as plt

print(history.history.keys())

plt.plot(history.history['accuracy'], color='b')
plt.plot(history.history['val_accuracy'], color='r')

plt.show()